In [ ]:
# 包含三种大模型api调用的测试

# 同一的函数调用
# create_modelname_request_message()
# fetch__modelname_model_response()

In [3]:
# 千帆平台 - 文心一言
# %pip install qianfan==0.3.9
import qianfan
from dotenv import load_dotenv, find_dotenv
from typing import Dict, List
_ = load_dotenv(find_dotenv())

def create_wenxin_request_messages(prompt: str) -> List[Dict[str, str]]:
    """
    构造用于文心模型的请求参数 `messages`。

    Args:
        prompt (str): 用户的输入提示词。

    Returns:
        List[Dict[str, str]]: 包含角色和内容的消息列表，用于文心模型的请求。
    """
    messages = [{"role": "user", 
                 "content": prompt}]
    return messages


def fetch_wenxin_model_response(prompt: str, 
                                model: str = "Yi-34B-Chat", 
                                temperature: float = 0.01) -> str:
    """
    从文心模型获取响应。

    Args:
        prompt (str): 输入提示词。
        model (str): 调用的模型名称，默认为 "Yi-34B-Chat"。也可以选择其他模型，如 "ERNIE-Bot-4"。
        temperature (float): 模型输出的温度系数，控制输出的随机程度。取值范围为 0 到 1.0，不可设置为 0。
                             温度系数越低，输出内容越一致。

    Returns:
        str: 文心模型的输出结果。

    Raises:
        ValueError: 如果模型调用返回错误或结果不符合预期。
    """
    # 实例化文心模型的 ChatCompletion 类
    chat_comp = qianfan.ChatCompletion()
    # 生成请求参数 messages
    message = create_wenxin_request_messages(prompt)

    # 执行模型调用
    resp = chat_comp.do(messages=message, 
                        model=model, 
                        temperature=temperature, 
                        # system = ,
                        penalty_score=1.5)
                        # penalty_score 
                        # 对已生成的token增加惩罚，减少重复生成的现象。
                        # 说明：（1）值越大表示惩罚越大。（2）取值范围：[1.0, 2.0]。

    # 检查返回结果，如果存在错误，则抛出异常
    if resp.get("code") != 200:
        raise ValueError(f"Model call returned an error: {resp.get('msg')}")

    # 返回模型的输出结果
    return resp["result"]

In [4]:
fetch_wenxin_model_response("你好，我是文心一言。")

'您好！很高兴见到您——作为一个人工智能助手，我随时准备帮助解答问题或提供信息支持。请问有什么我可以帮您的？'

In [6]:
# 讯飞平台 - 讯飞星火
# SDK方式调用
# pip install spark-ai-python0.3.15
import os

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
from sparkai.llm.llm import ChatSparkLLM, ChunkPrintHandler
from sparkai.core.messages import ChatMessage


def build_spark_config(model: str) -> dict:
    """
    根据指定的模型版本构造星火模型的请求配置。

    Args:
        model (str): 模型版本，支持 "v1.5"、"v2.0"、"v3.0" 和 "v3.5"。

    Returns:
        dict: 包含模型版本对应的域和API URL的配置字典。
    """
    spark_url_template = "wss://spark-api.xf-yun.com/{}/chat"
    model_configurations = {
        "v1.5": {"domain": "general", "spark_url": spark_url_template.format("v1.1")},
        "v2.0": {"domain": "generalv2", "spark_url": spark_url_template.format("v2.1")},
        "v3.0": {"domain": "generalv3", "spark_url": spark_url_template.format("v3.1")},
        "v3.5": {"domain": "generalv3.5", "spark_url": spark_url_template.format("v3.5")},
    }
    return model_configurations[model]

def create_spark_request_messages(prompt: str) -> list:
    """
    创建用于星火模型请求的消息列表。

    Args:
        prompt (str): 用户输入的提示词。

    Returns:
        list: 包含单个消息的列表，消息格式为字典，包含用户角色和内容。
    """
    messages = [ChatMessage(role="user", content=prompt)]
    return messages

def fetch_spark_model_response(prompt: str, 
                               model: str = "v3.5", 
                               temperature: float = 0.1) -> str:
    """
    调用星火模型并获取输出结果。

    Args:
        prompt (str): 输入的提示词。
        model (str): 要调用的模型版本，默认为 "v3.5"，也支持 "v3.0" 等其他版本。
        temperature (float): 模型输出的温度系数，控制输出的随机程度，取值范围为 0~1.0，不能为 0。

    Returns:
        str: 星火模型的调用结果。
    """
    spark_configuration = build_spark_config(model)
    
    spark_llm = ChatSparkLLM(
        spark_api_url=spark_configuration["spark_url"],
        spark_app_id=os.environ["SPARK_APPID"],
        spark_api_key=os.environ["SPARK_API_KEY"],
        spark_api_secret=os.environ["SPARK_API_SECRET"],
        spark_llm_domain=spark_configuration["domain"],
        temperature=temperature,
        streaming=False,
    )
    messages = create_spark_request_messages(prompt)
    handler = ChunkPrintHandler()
    response = spark_llm.generate([messages], callbacks=[handler])
    return response.generations[0][0].text

In [7]:
fetch_spark_model_response("你好")

'你好！有什么我能帮忙的吗？'

In [1]:
# 智谱GLM 
# %pip install zhipuai==2.0.1
import os
from dotenv import load_dotenv, find_dotenv
from zhipuai import ZhipuAI
_ = load_dotenv(find_dotenv())
client = ZhipuAI(api_key=os.getenv("ZHIPU_API_KEY"))
print(client)

def create_zhipuai_request_messages(prompt: str) -> list:
    """
    创建用于智谱GLM模型请求的消息列表。

    Args:
        prompt (str): 用户输入的提示词。

    Returns:
        list: 包含单个消息的列表，消息格式为字典，包含用户角色和内容。
    """
    messages = [{"role": "user", "content": prompt}]
    return messages


def fetch_zhipuai_model_response(prompt: str, 
                                 model: str = "glm-4", 
                                 temperature: float = 0.1) -> str:
    """
    调用智谱GLM模型并获取输出结果。

    Args:
        prompt (str): 输入的提示词。
        model (str): 要调用的模型名称，默认为 "zhipuai-glm"。
        temperature (float): 模型输出的温度系数，控制输出的随机程度，取值范围为 0~1.0，不能为 0。

    Returns:
        str: 智谱GLM模型的调用结果。
    """
    messages = create_zhipuai_request_messages(prompt)
    response = client.chat.completions.create(model=model,
                                              messages=messages,
                                              temperature=temperature)
    
    return response.choices[0].message.content

In [5]:
fetch_zhipuai_model_response("你好")

'你好👋！我是人工智能助手智谱清言，可以叫我小智🤖，很高兴见到你，欢迎问我任何问题。'